# Wiki Scrape of Previous Champs

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import html5lib

import requests 
from bs4 import BeautifulSoup
import urllib
import time

The url was read in, very similar to our other wikipedia scraper.

In [2]:
a = pd.read_html('https://en.wikipedia.org/wiki/List_of_UFC_champions', flavor = 'html5lib')

Each table was put in a dataframe and they were merged

In [3]:
a2 = a[2]
a3 = a[3]
a4 = a[4]
a5 = a[5]
a6 = a[6]
a7 = a[7]
a8 = a[8]
a9 = a[9]
a10= a[10]
a11 = a[11]
a12 = a[12]

fighters = pd.concat([a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12])

## Cleaning

The first row was once again changed to the column headers

In [4]:
fighters.columns = fighters.iloc[0]
fighters = fighters.drop(fighters.index[[0]])
fighters.reset_index(drop = True)

fighters.head(4)

,No.,Name,Event,Date,Reign(total),Defenses
1,1,Mark Colemandef. Dan Severn,"UFC 12Dothan, AL, US","Feb 7, 1997",170 days,NaN
2,2,Maurice Smith,"UFC 14Birmingham, AL, US","Jul 27, 1997",147 days,"1. def. Tank Abbott at UFC 15 on Oct 17, 1997"
3,3,Randy Couture,"UFC JapanYokohama, Japan","Dec 21, 1997",? days,NaN
4,Couture was stripped of the title in January 1...,NaN,NaN,NaN,NaN,NaN


These rows weren't relevant to our project and were subsequently dropped

In [5]:
to_drop = ['No.',
            'Event',
           'Date'
            ]
fighters.drop(to_drop, inplace=True, axis=1)
fighters = fighters[pd.notnull(fighters['Name'])]
fighters.head(5)

,Name,Reign(total),Defenses
1,Mark Colemandef. Dan Severn,170 days,NaN
2,Maurice Smith,147 days,"1. def. Tank Abbott at UFC 15 on Oct 17, 1997"
3,Randy Couture,? days,NaN
5,Bas Rutten def. Kevin Randleman,? days,NaN
7,Kevin Randlemandef. Pete Williams,364 days,"1. def. Pedro Rizzo at UFC 26 on Jun 9, 2000"


The "Name" column had who the fighter defeated to obtain the belt so that part had to be dropped.

In [6]:
fighters['Name'] = fighters['Name'].map(lambda x: x.split('def')[0].strip())
fighters['Name'] = fighters['Name'].map(lambda x: x.split('(')[0].strip())


"Defences" or title defences was cleaned by replacing Nan values with zeroes

In [7]:
fighters['Defenses'] = fighters['Defenses'].fillna('0')

In [8]:
fighters.head(5)

,Name,Reign(total),Defenses
1,Mark Coleman,170 days,0
2,Maurice Smith,147 days,"1. def. Tank Abbott at UFC 15 on Oct 17, 1997"
3,Randy Couture,? days,0
5,Bas Rutten,? days,0
7,Kevin Randleman,364 days,"1. def. Pedro Rizzo at UFC 26 on Jun 9, 2000"


Defences also had who the fighter defeated to obtain the belt so that was dropped also

In [9]:
fighters['Defenses'] = fighters['Defenses'].map(lambda x: x.split('.')[0].strip())
fighters['Name'] = fighters['Name'].map(lambda x: x.split('promoted')[0].strip())


Reign had to be cleaned. This was done by removing the word "days" and replacing question marks and dashes with zeroes.

In [10]:
fighters['Reign(total)'] = fighters['Reign(total)'].map(lambda x: x.split('days')[0].strip())
fighters['Reign(total)'] = fighters['Reign(total)'].map(lambda x: x.replace('?','0'))
fighters['Reign(total)'] = fighters['Reign(total)'].map(lambda x: x.replace('—','0'))
fighters.head(5)

,Name,Reign(total),Defenses
1,Mark Coleman,170,0
2,Maurice Smith,147,1
3,Randy Couture,0,0
5,Bas Rutten,0,0
7,Kevin Randleman,364,1


I inserted a new column callled 'ex_champ' and set all the values to true.

In [11]:
fighters.insert(3, "ex_champ", True)


In [12]:
fighters.head(5)

,Name,Reign(total),Defenses,ex_champ
1,Mark Coleman,170,0,True
2,Maurice Smith,147,1,True
3,Randy Couture,0,0,True
5,Bas Rutten,0,0,True
7,Kevin Randleman,364,1,True


Filled the Nan values in "ex_champ" with false

In [13]:
fighters['ex_champ'] = fighters['ex_champ'].fillna(False)


In [14]:
fighters.to_csv('ex_champ.csv',index = False)